# Scenario 2b: Collaboration and Compare - AML Run Analysis and Model comparison
Search runs with alpha > 0

1.	User A has access to User B’s training metrics and want to compare it their training results. OR User A has done a sweep job and wants to find the best model
2.	Analyze runs (metrics, parameters, etc) interactively 
3.	Use mlflow.search_runs() and other commands to load into Pandas dataframe and do common analysis (mlflow.search_runs())

    a.	compare job A to job B

    b.	find best model in sweep based on primary metric

    c.	find best model in sweep based on some other logged metric etc

Using [MLFlow Search API](https://mlflow.org/docs/latest/search-syntax.html#search) for the seardch queries below

### Compare Runs

In [3]:
import mlflow
from azureml.core import Workspace;
from mlflow.tracking.client import MlflowClient
from mlflow.entities import ViewType

mlflow.set_tracking_uri(Workspace.from_config().get_mlflow_tracking_uri())

all_experiments = [exp.experiment_id for exp in MlflowClient().list_experiments()]
query = "metrics.alpha > 0"
runs = mlflow.search_runs(experiment_ids=["c3732977-6d76-4192-bccf-38cac682b7b7", "c24a63cd-22b3-482c-818a-e532d96362f6"], filter_string=query, run_view_type=ViewType.ALL)

# print(runs)
runs.head(10)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.alpha,metrics.Test samples,metrics.Training samples,metrics.mse,tags.mlflow.source.name,tags.mlflow.user,tags.mlflow.source.type,tags.mlflow.rootRunId
0,783aebf9-16af-4aa3-b8d1-269624c2ac3f,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-10-01 18:45:06.462000+00:00,2021-10-01 18:45:07.199000+00:00,0.03,NaN,NaN,NaN,/anaconda/envs/azureml_py36/lib/python3.6/site...,azureuser,LOCAL,783aebf9-16af-4aa3-b8d1-269624c2ac3f
1,c2a8fa26-d5f4-4c60-b8fd-4465f2919ab0,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-10-01 19:02:47.519000+00:00,2021-10-01 19:02:48.108000+00:00,0.03,NaN,NaN,NaN,/anaconda/envs/azureml_py36/lib/python3.6/site...,azureuser,LOCAL,c2a8fa26-d5f4-4c60-b8fd-4465f2919ab0
2,experiment_with_mlflow_remote_1633124216_c95519e1,c24a63cd-22b3-482c-818a-e532d96362f6,FINISHED,,2021-10-01 21:38:34.242000+00:00,2021-10-01 21:40:25.009000+00:00,0.03,89.0,353.0,3424.900316,train.py,None,JOB,experiment_with_mlflow_remote_1633124216_c95519e1
3,396f7638-09b8-41ef-a520-0bd2be5814a6,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-03 00:54:28.231000+00:00,2021-11-03 00:54:29.228000+00:00,0.03,NaN,NaN,NaN,/anaconda/envs/azureml_py36/lib/python3.6/site...,azureuser,LOCAL,396f7638-09b8-41ef-a520-0bd2be5814a6
4,22930b3d-62f7-4ffb-aabb-f689058733ba,c3732977-6d76-4192-bccf-38cac682b7b7,FINISHED,,2021-11-16 00:13:06.239000+00:00,2021-11-16 00:13:07.243000+00:00,0.03,NaN,NaN,NaN,/anaconda/envs/azureml_py36/lib/python3.6/site...,azureuser,LOCAL,22930b3d-62f7-4ffb-aabb-f689058733ba


### List Models

In [4]:
from pprint import pprint

client = MlflowClient()
for rm in client.list_registered_models():
    pprint(dict(rm), indent=4)

{   'creation_timestamp': 1633124474397,
    'description': '',
    'last_updated_timestamp': 1633124474397,
    'latest_versions': [   <ModelVersion: creation_timestamp=1633124474429, current_stage='None', description='', last_updated_timestamp=1633124474429, name='my-model', run_id='experiment_with_mlflow_remote_1633124216_c95519e1', run_link='', source='', status='READY', status_message='', tags={}, user_id='', version='1'>],
    'name': 'my-model',
    'tags': {}}
{   'creation_timestamp': 1633728453446,
    'description': '',
    'last_updated_timestamp': 1633728453446,
    'latest_versions': [   <ModelVersion: creation_timestamp=1633728453472, current_stage='None', description='', last_updated_timestamp=1633728453472, name='4d525968-a1af-4503-968b-061a46855da5', run_id='', run_link='', source='', status='READY', status_message='', tags={}, user_id='', version='1'>],
    'name': '4d525968-a1af-4503-968b-061a46855da5',
    'tags': {}}
{   'creation_timestamp': 1635204959134,
    'd

## Query artifacts from a run (including Models) and Download

In [5]:
model = mlflow.sklearn.load_model("runs:/experiment_with_mlflow_remote_1633124216_c95519e1/model")

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Ridge from version 0.24.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
client = MlflowClient()

def print_artifact_info(artifact):
    print("artifact: {}".format(artifact.path))
    print("is_dir: {}".format(artifact.is_dir))
    print("size: {}".format(artifact.file_size))

#run
# Fetch the logged artifacts
artifacts = client.list_artifacts("experiment_with_mlflow_remote_1633124216_c95519e1")
for artifact in artifacts:
    #print_artifact_info(artifact)
    print(artifact)

client.download_artifacts("experiment_with_mlflow_remote_1633124216_c95519e1","model",".")

<FileInfo: file_size=-1, is_dir=False, path='actuals_vs_predictions.png'>
<FileInfo: file_size=-1, is_dir=True, path='azureml-logs'>
<FileInfo: file_size=-1, is_dir=True, path='azureml-logs'>
<FileInfo: file_size=-1, is_dir=True, path='azureml-logs'>
<FileInfo: file_size=-1, is_dir=True, path='azureml-logs'>
<FileInfo: file_size=-1, is_dir=True, path='azureml-logs'>
<FileInfo: file_size=-1, is_dir=True, path='azureml-logs'>
<FileInfo: file_size=-1, is_dir=True, path='logs'>
<FileInfo: file_size=-1, is_dir=True, path='logs'>
<FileInfo: file_size=-1, is_dir=True, path='logs'>
<FileInfo: file_size=-1, is_dir=True, path='model'>
<FileInfo: file_size=-1, is_dir=True, path='model'>
<FileInfo: file_size=-1, is_dir=True, path='model'>
<FileInfo: file_size=-1, is_dir=True, path='model'>


'/mnt/batch/tasks/shared/LS_root/mounts/clusters/osomorog4/code/Users/osomorog/MLFlow_Scenarios/Scenario2_Train_deploy_with_MLFlow_and_AML/Scenario2b_Search_and_Compare_with_MLFlow/model'